# Implementation of Feature Models on Instacart

In [8]:
import pandas as pd

PATH = "data/instacart/"
train = pd.read_csv(f"{PATH}order_products__train.csv")
prior = pd.read_csv(f"{PATH}order_products__prior.csv")
aisle = pd.read_csv(f"{PATH}aisles.csv")
dept = pd.read_csv(f"{PATH}departments.csv")
orders = pd.read_csv(f"{PATH}orders.csv")
prods = pd.read_csv(f"{PATH}products.csv")

## Data

orders
* order_id, user_id, order_number, order_dow, order_hour_of_day, days_since_prior_order

train, prior
* order_id, product_id, add_to_cart_order, reordered

aisle
* aisle_id, aisle

dept
* department_id, department

prods
* product_id, product_name, aisle_id, department_id


## Features

#TODO: Decide which features are parent/children